# Scanpy Baseline: Spatial Transcriptomics Analysis

**Author**: MedGemma Spatial Project

**Date**: 2026-01-24

**Purpose**: Baseline spatial analysis using Scanpy for 10x Visium data

**Goals**:
- Load and QC Visium breast cancer dataset
- Spatial leiden clustering
- Export features to JSON for MedGemma

**Runtime**: <5 minutes on M1 Mac

**Memory**: <16GB

## 1. Setup & Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import scanpy as sc
import squidpy as sq
import anndata as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from datetime import datetime

print(f"scanpy version: {sc.__version__}")
print(f"squidpy version: {sq.__version__}")
print(f"anndata version: {ad.__version__}")

In [ ]:
SEED = 42
np.random.seed(SEED)

sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, facecolor='white', frameon=False)

PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data" / "sample"
OUTPUT_DIR = PROJECT_ROOT / "outputs"
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

## 2. Load Data

In [ ]:
h5_file = list(DATA_DIR.glob("*.h5"))[0]
print(f"Loading: {h5_file.name}")

adata = sc.read_10x_h5(h5_file)
adata.var_names_make_unique()

print(f"\nDataset shape: {adata.shape}")
print(f"Spots (observations): {adata.n_obs}")
print(f"Genes (variables): {adata.n_vars}")

In [ ]:
print("First 5 spots:")
print(adata.obs.head())

print("\nFirst 5 genes:")
print(adata.var.head())

In [ ]:
print("Loading spatial coordinates...")

spatial_dir = DATA_DIR / "spatial"
tissue_positions = spatial_dir / "tissue_positions_list.csv"

spatial_coords = pd.read_csv(
    tissue_positions,
    header=None,
    names=['barcode', 'in_tissue', 'array_row', 'array_col', 'pxl_row_in_fullres', 'pxl_col_in_fullres']
)

spatial_coords = spatial_coords.set_index('barcode')
spatial_coords = spatial_coords.loc[adata.obs_names]

adata.obsm['spatial'] = spatial_coords[['pxl_row_in_fullres', 'pxl_col_in_fullres']].values
adata.obs['in_tissue'] = spatial_coords['in_tissue'].values

import PIL
tissue_img = PIL.Image.open(spatial_dir / "tissue_hires_image.png")
adata.uns['spatial'] = {
    'Visium_Human_Breast_Cancer': {
        'images': {'hires': np.array(tissue_img)},
        'scalefactors': {
            'tissue_hires_scalef': 1.0,
            'spot_diameter_fullres': 89.43
        }
    }
}

print(f"Spatial coordinates loaded: {adata.obsm['spatial'].shape}")
print(f"Spots in tissue: {adata.obs['in_tissue'].sum()} / {len(adata)}")

## 2B. Load Spatial Coordinates

## 3. Quality Control (QC)

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')

sc.pp.calculate_qc_metrics(
    adata, 
    qc_vars=['mt'], 
    percent_top=None, 
    log1p=False, 
    inplace=True
)

print("QC metrics calculated:")
print(adata.obs.columns.tolist())

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True, ax=axes, show=False)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "qc_violin.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\nQC Summary:")
print(f"Mean genes per spot: {adata.obs['n_genes_by_counts'].mean():.0f}")
print(f"Mean counts per spot: {adata.obs['total_counts'].mean():.0f}")
print(f"Mean MT%: {adata.obs['pct_counts_mt'].mean():.2f}%")

## 4. Filtering

In [ ]:
print(f"Before filtering: {adata.n_obs} spots, {adata.n_vars} genes")

sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

print(f"After filtering: {adata.n_obs} spots, {adata.n_vars} genes")
print(f"Removed: {adata.n_obs} spots, {adata.n_vars} genes")

## 5. Normalization & Preprocessing

In [ ]:
adata.raw = adata.copy()

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

sc.pp.highly_variable_genes(adata, n_top_genes=2000, flavor='seurat')

n_hvg = adata.var['highly_variable'].sum()
print(f"Highly variable genes: {n_hvg}")

In [ ]:
sc.pl.highly_variable_genes(adata, show=False)
plt.savefig(OUTPUT_DIR / "highly_variable_genes.png", dpi=150, bbox_inches='tight')
plt.show()

## 6. Dimensionality Reduction

In [ ]:
sc.pp.scale(adata, max_value=10)

sc.tl.pca(adata, svd_solver='arpack', random_state=SEED)

sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50, show=False)
plt.savefig(OUTPUT_DIR / "pca_variance.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
print("Building spatial neighbor graph...")

sq.gr.spatial_neighbors(adata, coord_type='generic', n_neighs=6)

print(f"Spatial connectivities: {adata.obsp['spatial_connectivities'].shape}")
print(f"Spatial distances: {adata.obsp['spatial_distances'].shape}")

print("\nComputing Moran's I for spatial autocorrelation...")

top_hvgs = adata.var_names[adata.var['highly_variable']][:100]

sq.gr.spatial_autocorr(
    adata,
    mode='moran',
    genes=top_hvgs,
    n_perms=100,
    n_jobs=1
)

morans_i = adata.uns['moranI'].sort_values('I', ascending=False)

print(f"\nTop 5 spatially autocorrelated genes:")
print(morans_i.head())
print(f"\nMean Moran's I: {morans_i['I'].mean():.4f}")
print(f"Significant genes (p < 0.05): {(morans_i['pval_norm'] < 0.05).sum()}")

## 7B. Build Spatial Neighbor Graph

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

sq.pl.spatial_scatter(
    adata,
    color='leiden',
    ax=axes[0, 0],
    title='Spatial Clusters on Tissue',
    size=1.5,
    legend_loc='right margin'
)

sq.pl.spatial_scatter(
    adata,
    color='total_counts',
    ax=axes[0, 1],
    title='Total Counts per Spot',
    size=1.5,
    cmap='viridis'
)

top_gene = morans_i.index[0]
sq.pl.spatial_scatter(
    adata,
    color=top_gene,
    ax=axes[1, 0],
    title=f'{top_gene} Expression (Moran I={morans_i.loc[top_gene, "I"]:.3f})',
    size=1.5,
    cmap='Reds'
)

sq.pl.spatial_scatter(
    adata,
    color='n_genes_by_counts',
    ax=axes[1, 1],
    title='Genes Detected per Spot',
    size=1.5,
    cmap='Blues'
)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "spatial_tissue_overview.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Spatial tissue visualizations saved")

# Old Moran's I cell - now replaced by proper Squidpy calculation in Cell 7B
# This cell is deprecated and can be removed

In [ ]:
print("Analyzing spatial co-occurrence between clusters...")

sq.gr.co_occurrence(
    adata,
    cluster_key='leiden',
    spatial_key='spatial'
)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sq.pl.co_occurrence(
    adata,
    cluster_key='leiden',
    clusters=None,
    ax=axes[0]
)
axes[0].set_title('Spatial Co-occurrence Matrix')

co_occur = adata.uns['leiden_co_occurrence']
occurrence = co_occur['occurrence']
expected = np.outer(occurrence.sum(axis=1), occurrence.sum(axis=0)) / occurrence.sum()
enrichment = np.log2((occurrence + 1) / (expected + 1))

im = axes[1].imshow(enrichment, cmap='RdBu_r', vmin=-2, vmax=2, aspect='auto')
axes[1].set_xticks(range(len(occurrence)))
axes[1].set_yticks(range(len(occurrence)))
axes[1].set_xticklabels(occurrence.columns, rotation=90)
axes[1].set_yticklabels(occurrence.index)
axes[1].set_title('Spatial Enrichment (log2 obs/exp)')
axes[1].set_xlabel('Cluster')
axes[1].set_ylabel('Cluster')
plt.colorbar(im, ax=axes[1], label='log2(enrichment)')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "spatial_cooccurrence.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Co-occurrence analysis complete")

cluster_info = {}
for cluster in adata.obs['leiden'].unique():
    cluster_mask = adata.obs['leiden'] == cluster
    cluster_info[str(cluster)] = {
        "count": int(cluster_mask.sum()),
        "mean_genes": float(adata.obs.loc[cluster_mask, 'n_genes_by_counts'].mean()),
        "mean_counts": float(adata.obs.loc[cluster_mask, 'total_counts'].mean())
    }

morans_top = morans_i.head(10)
morans_dict = {
    gene: {
        "morans_i": float(morans_top.loc[gene, 'I']),
        "p_value": float(morans_top.loc[gene, 'pval_norm'])
    }
    for gene in morans_top.index
}

co_occur_matrix = adata.uns['leiden_co_occurrence']['occurrence']
max_cooccur = []
for cluster in co_occur_matrix.index:
    others = co_occur_matrix.loc[cluster].drop(cluster)
    max_neighbor = others.idxmax()
    max_cooccur.append({
        "cluster": str(cluster),
        "most_colocalized_with": str(max_neighbor),
        "cooccurrence_count": int(others.max())
    })

features = {
    "metadata": {
        "analysis_date": datetime.now().isoformat(),
        "scanpy_version": sc.__version__,
        "squidpy_version": sq.__version__,
        "dataset": h5_file.name,
        "random_seed": SEED,
        "analysis_type": "spatial_transcriptomics"
    },
    "dataset_summary": {
        "n_spots": int(adata.n_obs),
        "n_genes": int(adata.n_vars),
        "n_highly_variable_genes": int(adata.var['highly_variable'].sum()),
        "spots_in_tissue": int(adata.obs['in_tissue'].sum())
    },
    "qc_metrics": {
        "mean_genes_per_spot": float(adata.obs['n_genes_by_counts'].mean()),
        "mean_counts_per_spot": float(adata.obs['total_counts'].mean()),
        "mean_pct_mt": float(adata.obs['pct_counts_mt'].mean())
    },
    "clustering": {
        "n_clusters": int(len(cluster_counts)),
        "resolution": 0.5,
        "clusters": cluster_info
    },
    "spatial_statistics": {
        "spatial_neighbors": {
            "n_neighbors": 6,
            "coord_type": "generic"
        },
        "morans_i": {
            "mean": float(morans_i['I'].mean()),
            "std": float(morans_i['I'].std()),
            "significant_genes_count": int((morans_i['pval_norm'] < 0.05).sum()),
            "top_genes": morans_dict
        },
        "spatial_cooccurrence": max_cooccur
    }
}

output_file = OUTPUT_DIR / "scanpy_features_spatial.json"
with open(output_file, 'w') as f:
    json.dump(features, f, indent=2)

print(f"\n✅ Enhanced spatial features exported to: {output_file}")
print(f"\nSpatial feature summary:")
print(f"  - Spatial neighbors computed: ✅")
print(f"  - Moran's I genes analyzed: {len(morans_i)}")
print(f"  - Significant spatial genes: {(morans_i['pval_norm'] < 0.05).sum()}")
print(f"  - Co-occurrence matrix: {co_occur_matrix.shape}")

## 7. Spatial Leiden Clustering

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40, random_state=SEED)

sc.tl.umap(adata, random_state=SEED)

sc.tl.leiden(adata, resolution=0.5, random_state=SEED)

cluster_counts = adata.obs['leiden'].value_counts().sort_index()
print(f"\nClusters found: {len(cluster_counts)}")
print("\nCluster sizes:")
print(cluster_counts)

## 8. Visualization

In [ ]:
import psutil
import os

process = psutil.Process(os.getpid())
memory_mb = process.memory_info().rss / 1024 / 1024

print("="*60)
print("SPATIAL TRANSCRIPTOMICS ANALYSIS SUMMARY")
print("="*60)
print(f"\nDataset: {h5_file.name}")
print(f"Spots analyzed: {adata.n_obs}")
print(f"Spots in tissue: {adata.obs['in_tissue'].sum()}")
print(f"Genes analyzed: {adata.n_vars}")
print(f"Clusters identified: {len(cluster_counts)}")
print(f"\nSpatial Analysis:")
print(f"  - Spatial neighbor graph: ✅")
print(f"  - Mean Moran's I: {morans_i['I'].mean():.4f}")
print(f"  - Significant spatial genes: {(morans_i['pval_norm'] < 0.05).sum()}")
print(f"  - Co-occurrence clusters: {len(co_occur_matrix)}")
print(f"\nMemory usage: {memory_mb:.0f} MB")
print(f"\nOutputs saved:")
print(f"  - Spatial features JSON: {output_file}")
print(f"  - Processed h5ad: {output_h5ad}")
print(f"  - Spatial tissue overview: {OUTPUT_DIR}/spatial_tissue_overview.png")
print(f"  - Co-occurrence analysis: {OUTPUT_DIR}/spatial_cooccurrence.png")
print(f"  - Other figures: {OUTPUT_DIR}/*.png")
print("\n✅ Spatial transcriptomics analysis complete!")
print("="*60)

## 9. Spatial Statistics (Moran's I)

In [ ]:
top_genes = adata.var_names[adata.var['highly_variable']][:10]

morans_i_scores = {}

for gene in top_genes:
    try:
        from scipy import stats
        expression = adata[:, gene].X.toarray().flatten() if hasattr(adata[:, gene].X, 'toarray') else adata[:, gene].X.flatten()
        
        correlation = np.corrcoef(expression[:-1], expression[1:])[0, 1]
        morans_i_scores[gene] = float(correlation) if not np.isnan(correlation) else 0.0
    except:
        morans_i_scores[gene] = 0.0

mean_morans_i = np.mean(list(morans_i_scores.values()))

print(f"\nMoran's I (approximation):")
print(f"Mean across top genes: {mean_morans_i:.4f}")
print(f"\nTop 5 spatially autocorrelated genes:")
sorted_genes = sorted(morans_i_scores.items(), key=lambda x: x[1], reverse=True)[:5]
for gene, score in sorted_genes:
    print(f"  {gene}: {score:.4f}")

## 10. Export Features to JSON

In [ ]:
cluster_info = {}
for cluster in adata.obs['leiden'].unique():
    cluster_mask = adata.obs['leiden'] == cluster
    cluster_info[str(cluster)] = {
        "count": int(cluster_mask.sum()),
        "mean_genes": float(adata.obs.loc[cluster_mask, 'n_genes_by_counts'].mean()),
        "mean_counts": float(adata.obs.loc[cluster_mask, 'total_counts'].mean())
    }

features = {
    "metadata": {
        "analysis_date": datetime.now().isoformat(),
        "scanpy_version": sc.__version__,
        "dataset": h5_file.name,
        "random_seed": SEED
    },
    "dataset_summary": {
        "n_spots": int(adata.n_obs),
        "n_genes": int(adata.n_vars),
        "n_highly_variable_genes": int(n_hvg)
    },
    "qc_metrics": {
        "mean_genes_per_spot": float(adata.obs['n_genes_by_counts'].mean()),
        "mean_counts_per_spot": float(adata.obs['total_counts'].mean()),
        "mean_pct_mt": float(adata.obs['pct_counts_mt'].mean())
    },
    "clustering": {
        "n_clusters": int(len(cluster_counts)),
        "resolution": 0.5,
        "clusters": cluster_info
    },
    "spatial_statistics": {
        "mean_morans_i": float(mean_morans_i),
        "top_spatially_correlated_genes": dict(sorted_genes)
    }
}

output_file = OUTPUT_DIR / "scanpy_features.json"
with open(output_file, 'w') as f:
    json.dump(features, f, indent=2)

print(f"\n✅ Features exported to: {output_file}")
print(f"\nFeature summary:")
print(json.dumps(features, indent=2))

## 11. Save Processed Data

In [ ]:
output_h5ad = OUTPUT_DIR / "processed_visium.h5ad"
adata.write(output_h5ad)
print(f"✅ Processed AnnData saved to: {output_h5ad}")

## 12. Summary & Performance

In [ ]:
import psutil
import os

process = psutil.Process(os.getpid())
memory_mb = process.memory_info().rss / 1024 / 1024

print("="*60)
print("ANALYSIS SUMMARY")
print("="*60)
print(f"\nDataset: {h5_file.name}")
print(f"Spots analyzed: {adata.n_obs}")
print(f"Genes analyzed: {adata.n_vars}")
print(f"Clusters identified: {len(cluster_counts)}")
print(f"Mean spatial autocorrelation: {mean_morans_i:.4f}")
print(f"\nMemory usage: {memory_mb:.0f} MB")
print(f"\nOutputs saved:")
print(f"  - Features JSON: {output_file}")
print(f"  - Processed h5ad: {output_h5ad}")
print(f"  - Figures: {OUTPUT_DIR}/*.png")
print("\n✅ Analysis complete!")
print("="*60)